<a href="https://colab.research.google.com/github/jorgemunozl/vla-test/blob/main/pi05-test/fourth_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fourth Test Lerobot Vanilla Text Generation

After test our sub task implementation with a fixed temperature, this time we visualize the distribution of more probable tokens, tweaking the temperature.

In [2]:
from huggingface_hub import login
login()

In [4]:
from transformers import AutoTokenizer
# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("google/paligemma-3b-pt-224",
                                          use_fast=False)

Loading tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [5]:
MODEL_ID = "lerobot/pi05_base"
device = "cuda" if torch.cuda.is_available() else "cpu"

PALIGEMMA_EOS_TOKEN = 1
max_decoding_steps = 40
temperature = 0.1

# -----------------------------------------------------------------------------
# FIX for AttributeError: 'GemmaRMSNorm' object has no attribute 'weight'
# The custom transformers branch uses adaRMS which removes the weight parameter,
# but the __repr__ method expects it. We monkey-patch it here.
from transformers.models.gemma import modeling_gemma

def fixed_extra_repr(self):
    if hasattr(self, "weight"):
        repr_str = f"{tuple(self.weight.shape)}, eps={self.eps}"
    else:
        repr_str = f"eps={self.eps}"
    if getattr(self, "dense", None) is not None:
        repr_str += f", adaptive=True, cond_dim={self.cond_dim}"
    return repr_str

modeling_gemma.GemmaRMSNorm.extra_repr = fixed_extra_repr
# -----------------------------------------------------------------------------

print("Loading policy...")
policy = PI05Policy.from_pretrained(MODEL_ID)
policy.model.eval()
policy.model.to(device)

Loading policy...


config.json: 0.00B [00:00, ?B/s]

Loading model from: lerobot/pi05_base


model.safetensors:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

✓ Loaded state dict from model.safetensors
Remapped: action_in_proj.bias -> model.action_in_proj.bias
Remapped: action_in_proj.weight -> model.action_in_proj.weight
Remapped: action_out_proj.bias -> model.action_out_proj.bias
Remapped: action_out_proj.weight -> model.action_out_proj.weight
Remapped: paligemma_with_expert.gemma_expert.lm_head.weight -> model.paligemma_with_expert.gemma_expert.lm_head.weight
Remapped: paligemma_with_expert.gemma_expert.model.layers.0.input_layernorm.dense.bias -> model.paligemma_with_expert.gemma_expert.model.layers.0.input_layernorm.dense.bias
Remapped: paligemma_with_expert.gemma_expert.model.layers.0.input_layernorm.dense.weight -> model.paligemma_with_expert.gemma_expert.model.layers.0.input_layernorm.dense.weight
Remapped: paligemma_with_expert.gemma_expert.model.layers.0.mlp.down_proj.weight -> model.paligemma_with_expert.gemma_expert.model.layers.0.mlp.down_proj.weight
Remapped: paligemma_with_expert.gemma_expert.model.layers.0.mlp.gate_proj.weigh

PI05Model(
  (paligemma_with_expert): PaliGemmaWithExpertModel(
    (paligemma): PaliGemmaForConditionalGeneration(
      (model): PaliGemmaModel(
        (vision_tower): SiglipVisionModel(
          (vision_model): SiglipVisionTransformer(
            (embeddings): SiglipVisionEmbeddings(
              (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
              (position_embedding): Embedding(256, 1152)
            )
            (encoder): SiglipEncoder(
              (layers): ModuleList(
                (0-26): 27 x SiglipEncoderLayer(
                  (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
                  (self_attn): SiglipAttention(
                    (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
                    (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
                    (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
                    (

In [6]:
import os
import cv2
import torch
import numpy as np

img_share_path = '/content'
img_name_list = ['top.png']
img_list = []

if img_share_path and os.path.exists(img_share_path):
    # Load images from path
    for img_name in img_name_list:
        img_path = os.path.join(img_share_path, img_name)
        if os.path.exists(img_path):
            img = cv2.imread(img_path)
            if img is not None:
                # Convert BGR to RGB
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img_list.append(img)


In [7]:
from lerobot.utils.constants import OBS_LANGUAGE_TOKENS, OBS_LANGUAGE_ATTENTION_MASK

batch = {}
if len(img_list) > 0:
    # Prepare image tensor: [H, W, C] -> [1, H, W, C]
    # Normalize to [0, 1] as expected by _preprocess_images (which then scales to [-1, 1])
    img_tensor = torch.from_numpy(img_list[0]).float() / 255.0
    img_tensor = img_tensor.unsqueeze(0) # Add batch dimension -> [1, H, W, C]

    batch["observation.images.base_0_rgb"] = img_tensor

# Process Language Prompt
# Note: PI05 automatically prepends image embeddings, so we DO NOT need an <image> token in the text.
# However, we ensure the prompt ends with a newline for better model performance.
prompt = "Put the fruits in the basket\nSub task: "
if not prompt.endswith("\n"):
    prompt += "\n"

print(f"Tokenizing prompt: '{prompt.strip()}'")
tokenized = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512
)

# Add to batch (converting mask to bool for compatibility)
batch[OBS_LANGUAGE_TOKENS] = tokenized["input_ids"]
batch[OBS_LANGUAGE_ATTENTION_MASK] = tokenized["attention_mask"].bool()

Tokenizing prompt: 'Put the fruits in the basket
Sub task:'


In [8]:
processed_images, img_masks = policy._preprocess_images(batch)

In [13]:
with torch.no_grad():
    # FIX: Ensure images are in (B, C, H, W) format
    # The error "expected input... to have 3 channels, but got 224" occurs because
    # the input was (B, H, W, C) but the model expects (B, C, H, W).
    fixed_processed_images = []
    for img in processed_images:
        # Check if shape is (B, H, W, C=3) instead of (B, C=3, H, W)
        if img.ndim == 4 and img.shape[-1] == 3 and img.shape[1] != 3:
            img = img.permute(0, 3, 1, 2)
        fixed_processed_images.append(img)

    # Retrieve tokens and masks from the batch dictionary and move them to the correct device
    tokens = batch[OBS_LANGUAGE_TOKENS].to(device)
    masks = batch[OBS_LANGUAGE_ATTENTION_MASK].to(device)

    # -------------------------------------------------------------------------
    # Manual Sampling Loop to inspect Top 5 Tokens
    # -------------------------------------------------------------------------
    import torch.nn.functional as F

    # 1. Embed prefix
    prefix_embs, prefix_pad_masks, prefix_att_masks = policy.model.embed_prefix(
        fixed_processed_images, img_masks, tokens, masks.bool()
    )

    # 2. Prepare masks
    # make_att_2d_masks is in global scope from previous cell execution
    prefix_att_2d_masks = make_att_2d_masks(prefix_pad_masks, prefix_att_masks)
    prefix_position_ids = torch.cumsum(prefix_pad_masks, dim=1) - 1
    prefix_att_2d_masks_4d = policy.model._prepare_attention_masks_4d(prefix_att_2d_masks)

    # 3. Initial Forward Pass
    lang_model = policy.model.paligemma_with_expert.paligemma.language_model
    lang_model.config._attn_implementation = "eager"

    embeddings, past_key_values = policy.model.paligemma_with_expert.forward(
        attention_mask=prefix_att_2d_masks_4d,
        position_ids=prefix_position_ids,
        past_key_values=None,
        inputs_embeds=[prefix_embs, None],
        use_cache=True,
    )

    last_token_embed = embeddings[0][:, -1, :]
    last_logits = policy.model.paligemma_with_expert.paligemma.lm_head(last_token_embed)

    batch_size = last_logits.shape[0]
    device = last_logits.device

    prefix_valid_length = torch.sum(prefix_pad_masks, dim=1)
    output_tokens = torch.zeros((batch_size, max_decoding_steps), dtype=torch.long, device=device)
    all_eos = torch.zeros(batch_size, dtype=torch.bool, device=device)

    running_attention_mask = prefix_pad_masks.clone()

    print("Generation Start...")

    # Autoregressive Loop
    for step in range(max_decoding_steps):
        # --- Inspect Probabilities ---
        probs_all = F.softmax(last_logits, dim=-1)
        top_probs, top_indices = torch.topk(probs_all, 20, dim=-1)

        print(f"\nStep {step}:")
        for b in range(batch_size):
            print(f"  Batch {b} Top 5:")
            for k in range(20):
                token_idx = top_indices[b, k].item()
                prob = top_probs[b, k].item()
                token_str = tokenizer.decode([token_idx])
                print(f"    {k+1}: '{token_str}' (p={prob:.4f})")
        # -----------------------------

        # Sample next token
        current_temp = 0.3 # Using 0.7 as in the original cell call
        if current_temp > 0.0:
            probs = F.softmax(last_logits / current_temp, dim=-1)
            token = torch.multinomial(probs, num_samples=1)
        else:
            token = torch.argmax(last_logits, dim=-1, keepdim=True)

        output_tokens[:, step] = token.squeeze(-1)

        # Check for EOS
        all_eos |= (token.squeeze(-1) == PALIGEMMA_EOS_TOKEN)
        if all_eos.all():
            break

        # Feed the new token back in the model
        next_token_embeds = lang_model.embed_tokens(token)
        position_ids = prefix_valid_length[:, None] + step

        new_mask = torch.ones((batch_size, 1), dtype=running_attention_mask.dtype, device=device)
        running_attention_mask = torch.cat([running_attention_mask, new_mask], dim=1)

        embeds_list, past_key_values = policy.model.paligemma_with_expert.forward(
            inputs_embeds=[next_token_embeds, None],
            attention_mask=running_attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            use_cache=True,
        )

        prefix_output = embeds_list[0]
        last_token_embed = prefix_output[:, -1, :]
        last_logits = policy.model.paligemma_with_expert.paligemma.lm_head(last_token_embed)

    print(f"\nOutput tokens shape: {output_tokens.shape}")
    print(f"Output tokens:\n{output_tokens}")

    # Decode the generated tokens
    print("\n" + "="*80)
    # The variable high_level_prompt is not defined. I'm removing it.
    # print("High Level Prompt\n",high_level_prompt)
    print("Decoded output:")
    print("="*80)

    for batch_idx in range(output_tokens.shape[0]):
        batch_tokens = output_tokens[batch_idx]
        # Remove padding (zeros) and decode
        non_zero_tokens = batch_tokens[batch_tokens != 0]
        if len(non_zero_tokens) > 0:
            decoded_text = tokenizer.decode(non_zero_tokens, skip_special_tokens=False)
            print(f"Batch {batch_idx}: {decoded_text}")
        else:
            print(f"Batch {batch_idx}: (empty)")

print("\n" + "="*80)
print("Test completed successfully!")
print("="*80)

Generation Start...

Step 0:
  Batch 0 Top 5:
    1: 'Yes' (p=0.1366)
    2: 'No' (p=0.1207)
    3: 'pick' (p=0.0421)
    4: 'no' (p=0.0367)
    5: 'Sub' (p=0.0255)
    6: 'place' (p=0.0249)
    7: 'yes' (p=0.0162)
    8: 'put' (p=0.0132)
    9: 'adjust' (p=0.0068)
    10: ' to' (p=0.0057)
    11: 'hand' (p=0.0053)
    12: 'move' (p=0.0050)
    13: '  ' (p=0.0048)
    14: '<eos>' (p=0.0047)
    15: ' in' (p=0.0033)
    16: ' the' (p=0.0032)
    17: ' out' (p=0.0029)
    18: ' on' (p=0.0026)
    19: 'return' (p=0.0025)
    20: ' ' (p=0.0023)

Step 1:
  Batch 0 Top 5:
    1: '<eos>' (p=0.7305)
    2: ' green' (p=0.0690)
    3: ' blue' (p=0.0544)
    4: '  ' (p=0.0133)
    5: ' red' (p=0.0072)
    6: ' the' (p=0.0062)
    7: ' organizer' (p=0.0051)
    8: ' water' (p=0.0048)
    9: 'Yes' (p=0.0041)
    10: ' orange' (p=0.0039)
    11: 'ⅸ' (p=0.0032)
    12: '
' (p=0.0026)
    13: ' pink' (p=0.0020)
    14: ' can' (p=0.0019)
    15: ' basket' (p=0.0018)
    16: '

' (p=0.0017)
    17: ' to